# AG2 example connecting to Sap Generative AI Hub via LiteLLM Proxy

## [How AG2 works](https://docs.ag2.ai/latest/)

## LLM access via LiteLLM Proxy
AG2 supports the [LiteLLM Proxy](https://docs.ag2.ai/latest/docs/user-guide/models/litellm-proxy-server/installation/) for access via OpenAI API calls.

## Installation
Follow details in [LiteLLM Proxy setup for SAP Gen. AI Hub](../proxy_set_up.ipynb)

In [9]:
%pip install "ag2[openai]"


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Set env variables
Add the following variables from the service key in a file called ".env" and put it in the same folder where you run the notebook:
```
LITELLM_PROXY_API_KEY=sk-1234
PROXY_BASE_URL=http://localhost:4000
```

## Run the CrewAI with LiteLLM and SAP LLMs

In [1]:
import os
from typing import Any

import litellm
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern
from dotenv import load_dotenv

/Users/vasilisa/PycharmProjects/litellm-agentic-examples/.venv/lib/python3.12/site-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


Load your credentials as environment variables.

In [2]:
litellm.use_litellm_proxy = True
load_dotenv()
api_base = os.getenv("PROXY_BASE_URL")
api_key = os.getenv("LITELLM_PROXY_API_KEY")

Set up the model with your proxy params

In [3]:
llm_config = LLMConfig(config_list={"model": "sap/gpt-4o", "base_url": api_base, "api_key": api_key})

Define the agent tools.

In [4]:
def get_weather(city: str) -> str:
    """Moke function"""
    city_normalized = city.lower().replace(" ", "")

    mock_weather_db = {
        "newyork": "The weather in New York is sunny with a temperature of 25°C.",
        "london": "It's cloudy in London with a temperature of 15°C.",
        "tokyo": "Tokyo is experiencing light rain and a temperature of 18°C.",
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return f"The weather in {city} is sunny with a temperature of 20°C."

Define the function for check if agent conversation is finished

In [5]:
def is_termination_msg(msg: dict[str, Any]) -> bool:
    content = msg.get("content", "")
    return (content is not None) and "==== REPORT GENERATED ====" in content

Define the Agent with the SAP LLM and the tool.

In [6]:
assistant = ConversableAgent(name="assistant",
                             llm_config=llm_config,
                             system_message="You are a helpful weather assistant. "
                                            "When the user asks for the weather in a specific city, "
                                            "use the 'get_weather' tool to find the information. "
                                            "If the tool returns an error, inform the user politely. "
                                            "If the tool is successful, write a couple sentences for a "
                                            "TV weather report in the city, that will be include small joke."
                                            "Once you've generated the report append this to the summary:"
                                            "==== REPORT GENERATED ====",
                             functions=[get_weather])

Set up the conversation pattern with the agent, model and function for conversation termination

In [7]:
pattern = AutoPattern(initial_agent=assistant,
                      agents=[assistant],
                      group_manager_args={
                          "llm_config": llm_config,
                          "is_termination_msg": is_termination_msg
                      },
                      )

Run the conversation with message

In [8]:
result, _, _ = initiate_group_chat(pattern=pattern,
                                   messages="What is the weather like in Tbilisi?",
                                   )

_User (to chat_manager):

What is the weather like in Tbilisi?

--------------------------------------------------------------------------------

Next speaker: assistant


>>>>>>>> USING AUTO REPLY...
assistant (to chat_manager):


***** Suggested tool call (call_Aw8yVKzSOju1DD0zuDMZs8ZT): get_weather *****
Arguments: 
{"city":"Tbilisi"}
****************************************************************************

--------------------------------------------------------------------------------

Next speaker: _Group_Tool_Executor


>>>>>>>> EXECUTING FUNCTION get_weather...
Call ID: call_Aw8yVKzSOju1DD0zuDMZs8ZT
Input arguments: {'city': 'Tbilisi'}

>>>>>>>> EXECUTED FUNCTION get_weather...
Call ID: call_Aw8yVKzSOju1DD0zuDMZs8ZT
Input arguments: {'city': 'Tbilisi'}
Output:
The weather in Tbilisi is sunny with a temperature of 20°C.
_Group_Tool_Executor (to chat_manager):

***** Response from calling tool (call_Aw8yVKzSOju1DD0zuDMZs8ZT) *****
The weather in Tbilisi is sunny with a temper